In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import stats
import glob
import os
from natsort import natsorted
from util import *
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
vid_name = 'seq18'
path = 'viz_predictions/'+vid_name
savepath = 'results/'+vid_name
output_name = savepath+'/'+vid_name+'.json'

if not os.path.exists(savepath):
    os.makedirs(savepath)


In [3]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    return np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0))

In [4]:
#Marking the boudning boxes
for image in natsorted(glob.glob(path+'/rgb/'+'*.jpg')):

    frame = os.path.splitext(os.path.basename(image))[0]
    img = cv2.imread(path+'/gray/'+os.path.split(image)[1], 0)
    img_rgb = cv2.imread(path+'/rgb/'+os.path.split(image)[1], 1)
    keypoints = np.load(path+'/keypoints/'+frame+'.npy')
    pose = np.load(path+'/pose/'+frame+'.npy') #pose keypoint numpy array
    handloc = keypoints[:,:,:,0:2].astype(int)

    #Dictionary of keypoinnt and index
    part_loc = {'left':0, 'right':1, 'right_elbow':3, 'right_wrist':4, 'left_elbow':6, 'left_wrist':7}

    num_ppl = keypoints.shape[1]

    #Extracting depth values of each hand named to create hand 3D position as hands3D
    depth = img[handloc[:,:,:,1], handloc[:,:,:,0]]
    depth = depth.reshape(depth.shape[0],depth.shape[1],depth.shape[2],1)
    hands3D = np.concatenate([handloc[:,:,:,0:2], depth],axis=3)

    #-------------------------------------------------------------------------------------------------------------#

    #Adjust hadnkeypoint depth using elbow and wrist depth values
    hands3D_ = hands3D.copy()

    rightarm = pose[:, 3:5, 0:2].astype(int)
    leftarm = pose[:, 6:8, 0:2].astype(int)

    Rarm_depth = img[rightarm[:,:,1], rightarm[:,:,0]]
    Larm_depth = img[leftarm[:,:,1], leftarm[:,:,0]]
    Rdelta = Rarm_depth.mean(axis=1).astype(int).reshape(pose.shape[0],1)
    Ldelta = Larm_depth.mean(axis=1).astype(int).reshape(pose.shape[0],1)

    hands3D_[part_loc['left'],:,:,2] = (hands3D[part_loc['left'],:,:,2] + Ldelta)/2
    hands3D_[part_loc['right'],:,:,2] = (hands3D[part_loc['right'],:,:,2] + Rdelta)/2

    hands3D = hands3D_.copy()

    #-----------------------------------------------------------------------------------------------------------#

    #Obtain vectors along hands to determine angle
    Lvec = [1,-1]*unit_vector(leftarm[:,1] - leftarm[:,0])
    Rvec = [1,-1]*unit_vector(rightarm[:,1] - rightarm[:,0])

    #----------------------------------------------------------------------------------------------------------#

    start_pt =[]
    end_pt = []


    for i in range(num_ppl):
        for j in range(num_ppl):
            if i != j:
                diff1 = abs(hands3D[0,i,:,:] - hands3D[0,j,:,:])
                eucdis1 = np.linalg.norm(diff1, axis=1)
                mode1, count1 = stats.mode(eucdis1)

                ang_rad1 = angle_between(Lvec[i],Lvec[j])
                angle1 = ang_rad1*180/np.pi

                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis1) < 180 and sum(eucdis1) != 0 and angle1>90:
#                     print('Handshake1 detected. RUN!!!')
#                     print(angle1)
                    xs1 = np.argmin(np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1])))
                    ys1 = np.argmin(np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0])))
                    xe1 = np.argmax(np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1])))
                    ye1 = np.argmax(np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0])))
                    start1 = int(np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0]))[ys1]), int(np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1]))[xs1])
                    end1 = int(np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0]))[ye1]), int(np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1]))[xe1])

                    #Check if same points are already located
                    if len(start_pt) == 0:
                        start_pt.append(start1)
                        end_pt.append(end1)
                    else:
                        for tup in start_pt:
                            if tup==start1:
                                pass
                            else:
                                start_pt.append(start1)
                                end_pt.append(end1)

                diff2 = abs(hands3D[0,i,:,:] - hands3D[1,j,:,:])
                eucdis2 = np.linalg.norm(diff2, axis=1)
                mode2, count2 = stats.mode(eucdis2)

                ang_rad2 = angle_between(Lvec[i],Rvec[j])
                angle2 = ang_rad2*180/np.pi


                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis2) < 180 and sum(eucdis2) != 0 and angle2>90:
#                     print('Handshake2 detected. RUN!!!')
#                     print(angle2)
                    xs2 = np.argmin(np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1])))
                    ys2 = np.argmin(np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0])))
                    xe2 = np.argmax(np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1])))
                    ye2 = np.argmax(np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0])))
                    start2 = int(np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0]))[ys2]), int(np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1]))[xs2])
                    end2 = int(np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0]))[ye2]), int(np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1]))[xe2])

                    #Check if same points are already located
                    if len(start_pt) == 0:
                        start_pt.append(start2)
                        end_pt.append(end2)
                    else:
                        for tup in start_pt:
                            if tup==start2:
                                pass
                            else:
                                start_pt.append(start2)
                                end_pt.append(end2)

                diff3 = abs(hands3D[1,i,:,:] - hands3D[0,j,:,:])
                eucdis3 = np.linalg.norm(diff3, axis=1)
                mode3, count3 = stats.mode(eucdis3)

                ang_rad3 = angle_between(Rvec[i],Lvec[j])
                angle3 = ang_rad3*180/np.pi


                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis3) < 180 and sum(eucdis3) != 0 and angle3>90:
#                     print('Handshake3 detected. RUN!!!')
#                     print(angle3)
                    xs3 = np.argmin(np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1])))
                    ys3 = np.argmin(np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0])))
                    xe3 = np.argmax(np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1])))
                    ye3 = np.argmax(np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0])))
                    start3 = int(np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0]))[ys3]), int(np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1]))[xs3])
                    end3 = int(np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0]))[ye3]), int(np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1]))[xe3])

                    #Check if same points are already located
                    if len(start_pt) == 0:
                        start_pt.append(start3)
                        end_pt.append(end3)
                    else:
                        for tup in start_pt:
                            if tup==start3:
                                pass
                            else:
                                start_pt.append(start3)
                                end_pt.append(end3)

                diff4 = abs(hands3D[1,i,:,:] - hands3D[1,j,:,:])
                eucdis4 = np.linalg.norm(diff4, axis=1)
                mode4, count4 = stats.mode(eucdis4)

                ang_rad4 = angle_between(Rvec[i],Rvec[j])
                angle4 = ang_rad4*180/np.pi


                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis4) < 180 and sum(eucdis4) != 0 and angle4>90:
#                     print('Handshake4 detected. RUN!!!')
#                     print(angle4)
                    xs4 = np.argmin(np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1])))
                    ys4 = np.argmin(np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0])))
                    xe4 = np.argmax(np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1])))
                    ye4 = np.argmax(np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0])))
                    start4 = int(np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0]))[ys4]), int(np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1]))[xs4])
                    end4 = int(np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0]))[ye4]), int(np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1]))[xe4])

                    #Check if same points are already located
                    if len(start_pt) == 0:
                        start_pt.append(start4)
                        end_pt.append(end4)
                    else:
                        for tup in start_pt:
                            if tup==start4:
                                pass
                            else:
                                start_pt.append(start4)
                                end_pt.append(end4)

    #Output json file of keypoints
    init_json(output_name)

    json_data = {
        "filename" : vid_name,
        frame:{}
    }


    for ind in range(len(start_pt)):
        json_data[frame][ind] = {"shake_id": ind, "start_pt": start_pt[ind], "end_pt": end_pt[ind]}

    update_json(json_data, output_name)


File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already exists : results/seq18/seq18.json
File already 

IndexError: index 514 is out of bounds for axis 1 with size 512